# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.120828e+02     1.542893e+00
 * time: 0.7485859394073486
     1     1.129945e+01     9.433763e-01
 * time: 3.143934965133667
     2    -1.151691e+01     9.972489e-01
 * time: 3.7984039783477783
     3    -3.406944e+01     7.653398e-01
 * time: 4.793399810791016
     4    -4.755206e+01     6.028549e-01
 * time: 5.748171806335449
     5    -5.700028e+01     2.008281e-01
 * time: 6.717785835266113
     6    -5.980841e+01     1.524335e-01
 * time: 7.382828950881958
     7    -6.089081e+01     5.093624e-02
 * time: 8.058369874954224
     8    -6.131163e+01     5.386747e-02
 * time: 8.700815916061401
     9    -6.157358e+01     5.097503e-02
 * time: 9.345285892486572
    10    -6.178826e+01     3.082349e-02
 * time: 10.00771188735962
    11    -6.196061e+01     2.269800e-02
 * time: 10.651080846786499
    12    -6.204559e+01     2.173931e-02
 * time: 11.296340942382812
    13    -6.212062e+01     1.878717e-02
 * time: 11.93965482711792
   

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671068
    AtomicLocal         -18.8557657
    AtomicNonlocal      14.8522626
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485370 
    Xc                  -19.3336820

    total               -62.261666461280
